### Season 4 Phenotype Table
#### Started November 2019 after GenoPhenoEnvo Kickoff
#### Columns
* `range`
* `column`
* `canopy_height`
* `max_canopy_height`
* `days_to_` multiple values

In [2]:
# %pwd
%cd '/Users/ejcain/Dev/UA-AG/phenotypes/terraref-datasets/'

/Users/ejcain/Dev/UA-AG/phenotypes/terraref-datasets


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

* Season 4 data queried from betydb using R, then downloaded as csv
* Some columns have already been dropped in the very messy season_4_transformation notebook

In [3]:
df = pd.read_csv('more_traits_after_feedback.csv')
print(df.shape)
df.head()

(145623, 14)


,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373


In [17]:
# df.trait.astype('object').combine(df['mean'], func=)    

In [16]:
# new_df = df.assign(ambient_humidity=" ")


for index, row in new_df.iterrows():
    if row['trait'] == 'ambient_humidity':
        new_df.loc[index, ['ambient_humidity']] = row['mean']
        
new_df.head()b

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar,ambient_humidity
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355,26.5059
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355,
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355,
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373,20.5225
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373,


In [19]:
# this works
    
for index, row in new_df.iterrows():
    if row['trait'] == 'ambient_humidity':
        new_df.loc[index, ['ambient_humidity']] = row['mean']
        
new_df.head()

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,mean,units,cultivar,ambient_humidity
0,6000005593,6002011659,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,26.505859,%,PI511355,26.5059
1,6000005593,6002011662,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,43.840000,C,PI511355,
2,6000005593,6002011666,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,Difference between the leaf temperature and th...,-2.800000,C,PI511355,
3,6000005361,6002012219,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,Ambient relative humidity near the plant canopy,20.522461,%,PI570373,20.5225
4,6000005361,6002012222,MAC Field Scanner Season 4 Range 21 Column 2,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,Air temperature measured by handheld sensor ne...,44.189999,C,PI570373,


In [23]:
# will not scale, but will work for now



# for trait in trait_columns:
    
# for index, row in new_df.iterrows():
#     if row['trait'] == 'aboveground_dry_biomass':
#         new_df.loc[index, ['aboveground_dry_biomass']] = row['mean']
        
# new_df.head()

#### New Trait Columns

In [24]:
new_df.trait.unique()

array(['ambient_humidity', 'proximal_air_temperature',
       'leaf_temperature_differential', 'surface_temperature',
       'aboveground_dry_biomass', 'seedling_emergence_rate',
       'canopy_height', 'flag_leaf_emergence_time', 'flowering_time',
       'grain_stage_time', 'canopy_cover', 'emergence_count'],
      dtype=object)

Change column name `mean` due to being a keyword and also confusing

In [28]:
df = new_df.rename({'mean': 'value'}, axis=1)
df.tail()

,site_id,id,sitename,lat,lon,cultivar_id,date,time,raw_date,trait,trait_description,value,units,cultivar,ambient_humidity
145618,6000005865,6004797282,MAC Field Scanner Season 4 Range 41 Column 10,33.075985,-111.974901,6000000969,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",392.0,cm,PI573193,
145619,6000005866,6004797283,MAC Field Scanner Season 4 Range 41 Column 11,33.075985,-111.974884,6000000688,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",386.0,cm,PI152860,
145620,6000005882,6004797287,MAC Field Scanner Season 4 Range 41 Column 15,33.075985,-111.974819,6000000953,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",342.0,cm,PI570087,
145621,6000005906,6004797288,MAC Field Scanner Season 4 Range 42 Column 2,33.076021,-111.975032,6000000751,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",331.0,cm,PI257600,
145622,6000005888,6004797290,MAC Field Scanner Season 4 Range 42 Column 4,33.076021,-111.974999,6000000225,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",370.0,cm,PI154844,


In [29]:
df.ambient_humidity.nunique()

1254

In [30]:
df.shape

(145623, 15)

Drop `trait_description` and `units`

In [31]:
df.drop(labels='trait_description', axis=1, inplace=True)
df.drop(labels='units', axis=1, inplace=True)

In [32]:
df.shape

(145623, 13)

In [33]:
df.isnull().sum()

site_id             0
id                  0
sitename            0
lat                 0
lon                 0
cultivar_id         0
date                0
time                0
raw_date            0
trait               0
value               0
cultivar            0
ambient_humidity    0
dtype: int64

* Set sitename as index
* 
*

In [37]:
df = df.set_index(keys='sitename')
print(df.shape)
df.head()

(145623, 12)


,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity
sitename,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355,26.5059
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355,
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,PI570373,20.5225
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,PI570373,


In [38]:
df.dtypes

site_id               int64
id                    int64
lat                 float64
lon                 float64
cultivar_id           int64
date                 object
time                 object
raw_date             object
trait                object
value               float64
cultivar             object
ambient_humidity     object
dtype: object

#### Add `column` and `range` columns for easier location identification

In [39]:
df.head()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity
sitename,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355,26.5059
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355,
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,PI570373,20.5225
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,PI570373,


In [41]:
sitename_values = df.index.values
print(len(df))
print(df.index[:10])

145623
Index(['MAC Field Scanner Season 4 Range 26 Column 15',
       'MAC Field Scanner Season 4 Range 26 Column 15',
       'MAC Field Scanner Season 4 Range 26 Column 15',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2',
       'MAC Field Scanner Season 4 Range 21 Column 2'],
      dtype='object', name='sitename')


In [42]:
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))
print(no_e_w_names[:10])

145623
['MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 26 Column 15', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2', 'MAC Field Scanner Season 4 Range 21 Column 2']


In [43]:
# this cell should have no output

for name in no_e_w_names:
    
    if name.endswith(' W'):
        print(name)
        
    if name.endswith(' E'):
        print(name)
        
    if name.endswith(' '):
        print('This is wrong too.')

In [44]:
test_sitename_string = no_e_w_names[42]
test_sitename_string

'MAC Field Scanner Season 4 Range 26 Column 14'

In [45]:
split_string = test_sitename_string.split(' ')

In [46]:
print(split_string[6])
print(split_string[8])

26
14


In [47]:
ranges = []
columns = []

for name in no_e_w_names:
    
    split_name = name.split(' ')
    
    range_value = split_name[6]
    ranges.append(range_value)
    
    column_value = split_name[8]
    columns.append(column_value)
    
print(f'There are {len(ranges)} range values!')
print(f'First three range values: {ranges[:3]}')

print(f'There are {len(columns)} column values!')
print(f'Here are the first three column values! {columns[:3]}')

print(f'All of these numbers should be {len(sitename_index)}')

There are 145623 range values!
First three range values: ['26', '26', '26']
There are 145623 column values!
Here are the first three column values! ['15', '15', '15']
All of these numbers should be 145623


In [48]:
df.head(3)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity
sitename,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355,26.5059
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,PI511355,


In [49]:
df['range_location'] = ranges
df['column_location'] = columns

df.tail()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity,range_location,column_location
sitename,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 41 Column 10,6000005865,6004797282,33.075985,-111.974901,6000000969,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,392.0,PI573193,,41,10
MAC Field Scanner Season 4 Range 41 Column 11,6000005866,6004797283,33.075985,-111.974884,6000000688,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,386.0,PI152860,,41,11
MAC Field Scanner Season 4 Range 41 Column 15,6000005882,6004797287,33.075985,-111.974819,6000000953,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,342.0,PI570087,,41,15
MAC Field Scanner Season 4 Range 42 Column 2,6000005906,6004797288,33.076021,-111.975032,6000000751,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,331.0,PI257600,,42,2
MAC Field Scanner Season 4 Range 42 Column 4,6000005888,6004797290,33.076021,-111.974999,6000000225,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,370.0,PI154844,,42,4


In [50]:
# check that `E` and `W` columns have accurate range and column values

df.loc[df.index.str.endswith(' E')].sample(n=7)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity,range_location,column_location
sitename,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 41 Column 7 E,6000008236,6001941424,33.075985,-111.974946,6000000796,2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,59.00,PI329584,,41,7
MAC Field Scanner Season 4 Range 34 Column 14 E,6000008026,6001945219,33.075734,-111.974831,6000000834,2017 Jun 8 (America/Phoenix),[time unspecified or unknown],2017-06-09 00:00:00 -0500,canopy_height,138.00,PI452619,,34,14
MAC Field Scanner Season 4 Range 18 Column 6 E,6000007498,6001942179,33.075159,-111.974962,6000000739,2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,83.00,PI217691,,18,6
MAC Field Scanner Season 4 Range 47 Column 7 E,6000008428,6001942995,33.076201,-111.974946,6000000897,2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,135.00,PI563196,,47,7
MAC Field Scanner Season 4 Range 10 Column 12 E,6000007254,6001943373,33.074871,-111.974864,6000000575,2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,102.00,PI329501,,10,12
MAC Field Scanner Season 4 Range 14 Column 8 E,6000007374,6001943495,33.075015,-111.974929,6000000841,2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,167.00,PI455307,,14,8
MAC Field Scanner Season 4 Range 7 Column 9 E,6000007152,6001950920,33.074763,-111.974913,6000000879,2017 May 4 (America/Phoenix),[time unspecified or unknown],2017-05-05 00:00:00 -0500,seedling_emergence_rate,0.95,PI562897,,7,9


In [52]:
# check W's

df.loc[df.index.str.endswith(' W')].sample(n=4)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity,range_location,column_location
sitename,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 19 Column 6 W,6000007529,6001945606,33.075195,-111.974970,6000000821,2017 Jun 26 (America/Phoenix),[time unspecified or unknown],2017-06-27 00:00:00 -0500,canopy_height,224.0,PI330185,,19,6
MAC Field Scanner Season 4 Range 12 Column 15 W,6000007323,6001945839,33.074943,-111.974823,6000000216,2017 Jul 2 (America/Phoenix),[time unspecified or unknown],2017-07-03 00:00:00 -0500,canopy_height,240.0,PI641825,,12,15
MAC Field Scanner Season 4 Range 20 Column 8 W,6000007565,6001954955,33.075231,-111.974937,6000001021,2017 May 17 (America/Phoenix),[time unspecified or unknown],2017-05-18 00:00:00 -0500,canopy_cover,12.2,PI655972,,20,8
MAC Field Scanner Season 4 Range 34 Column 5 W,6000008007,6001944096,33.075734,-111.974987,6000000697,2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,167.0,PI153877,,34,5


In [86]:
# sitename_index.to_csv('column_and_range_df.csv')

In [53]:
df.shape

(145623, 14)

#### Create empty DataFrame with trait columns to merge with df

In [54]:
trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
                'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
                'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
                'emergence_count']

empty_df = pd.DataFrame(index=df.index, columns=trait_columns)

In [55]:
empty_df.head()

,proximal_air_temperature,leaf_temperature_differential,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flag_leaf_emergence_rate,flowering_time,grain_stage_time,canopy_cover,emergence_count
sitename,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
empty_df.shape

(145623, 11)

In [59]:
df.head(2)

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,cultivar,ambient_humidity,range_location,column_location
sitename,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,PI511355,26.5059,26,15
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,PI511355,,26,15


In [66]:
# empty_df.to_csv('empty_df_with_index.csv')
# df.to_csv('df_to_merge_with_df.csv')

In [73]:
print(empty_df.shape)
print(df.shape)

(145623, 11)
(145623, 14)


In [82]:
# check that the indices are identical

print(pd.Series(empty_df.index.isin(df.index)).value_counts())

True    145623
dtype: int64


In [83]:
what_is_happening = pd.concat([df, empty_df.reindex(df.index)], axis=1)
what_is_happening.shape

(145623, 25)

In [84]:
# the key for a successful merge / join / concat is to use an improper name for the resulting df

In [85]:
what_is_happening.head()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,...,leaf_temperature_differential,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flag_leaf_emergence_rate,flowering_time,grain_stage_time,canopy_cover,emergence_count
sitename,,,,,,,,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
what_is_happening.columns

Index(['site_id', 'id', 'lat', 'lon', 'cultivar_id', 'date', 'time',
       'raw_date', 'trait', 'value', 'cultivar', 'ambient_humidity',
       'range_location', 'column_location', 'proximal_air_temperature',
       'leaf_temperature_differential', 'surface_temperature',
       'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height',
       'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time',
       'canopy_cover', 'emergence_count'],
      dtype='object')

In [87]:
new_df = what_is_happening.copy()

In [89]:
new_df.to_csv('merged_df_0.csv')

#### _Now_ populate trait columns

In [91]:
trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
                'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
                'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
                'emergence_count']

for trait in trait_columns:

    for index, row in new_df.iterrows():
        if row['trait'] == trait:
            new_df.loc[index, [trait]] = row['value']
        
new_df.head()

,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,value,...,leaf_temperature_differential,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flag_leaf_emergence_rate,flowering_time,grain_stage_time,canopy_cover,emergence_count
sitename,,,,,,,,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,26.505859,...,-8.59,36.9336,NaN,NaN,348,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,43.840000,...,-8.59,36.9336,NaN,NaN,348,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,-2.800000,...,-8.59,36.9336,NaN,NaN,348,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,20.522461,...,1.92,35.3271,NaN,NaN,284,NaN,NaN,NaN,NaN,NaN
MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,44.189999,...,1.92,35.3271,NaN,NaN,284,NaN,NaN,NaN,NaN,NaN


In [92]:
new_df.to_csv('lots_of_traits_df.csv')

******

In [4]:
df = pd.read_csv('lots_of_traits_df.csv')
df.head()

,sitename,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,...,leaf_temperature_differential,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flag_leaf_emergence_rate,flowering_time,grain_stage_time,canopy_cover,emergence_count
0,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
1,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
2,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
3,MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,...,1.920002,35.327081,NaN,NaN,284.0,NaN,NaN,NaN,NaN,NaN
4,MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,...,1.920002,35.327081,NaN,NaN,284.0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(145623, 26)

In [6]:
df.isnull().sum()

sitename                              0
site_id                               0
id                                    0
lat                                   0
lon                                   0
cultivar_id                           0
date                                  0
time                                  0
raw_date                              0
trait                                 0
value                                 0
cultivar                              0
ambient_humidity                      0
range_location                        0
column_location                       0
proximal_air_temperature         137773
leaf_temperature_differential    137773
surface_temperature                5219
aboveground_dry_biomass          109832
seedling_emergence_rate          144548
canopy_height                     10740
flag_leaf_emergence_rate         145623
flowering_time                   136237
grain_stage_time                 137231
canopy_cover                     145522


Are all these value counts accounted for?!

In [7]:
df.trait.value_counts()

surface_temperature              80296
canopy_height                    58056
ambient_humidity                  2008
leaf_temperature_differential     2008
proximal_air_temperature          2008
emergence_count                    566
seedling_emergence_rate            279
aboveground_dry_biomass            200
flag_leaf_emergence_time            79
flowering_time                      54
grain_stage_time                    48
canopy_cover                        21
Name: trait, dtype: int64

Correct `flag_leaf_emergence_time` column

In [16]:
df = df.assign(flag_leaf_emergence_time = np.nan)
df.tail()

,sitename,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,...,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flag_leaf_emergence_rate,flowering_time,grain_stage_time,canopy_cover,emergence_count,flag_leaf_emergence_time
145618,MAC Field Scanner Season 4 Range 41 Column 10,6000005865,6004797282,33.075985,-111.974901,6000000969,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,...,35.671442,NaN,NaN,392.0,NaN,NaN,NaN,NaN,NaN,NaN
145619,MAC Field Scanner Season 4 Range 41 Column 11,6000005866,6004797283,33.075985,-111.974884,6000000688,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,...,35.646936,NaN,NaN,386.0,NaN,NaN,NaN,NaN,NaN,NaN
145620,MAC Field Scanner Season 4 Range 41 Column 15,6000005882,6004797287,33.075985,-111.974819,6000000953,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,...,36.943689,NaN,NaN,342.0,NaN,NaN,NaN,NaN,NaN,NaN
145621,MAC Field Scanner Season 4 Range 42 Column 2,6000005906,6004797288,33.076021,-111.975032,6000000751,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,...,34.652429,NaN,NaN,331.0,NaN,NaN,NaN,NaN,NaN,NaN
145622,MAC Field Scanner Season 4 Range 42 Column 4,6000005888,6004797290,33.076021,-111.974999,6000000225,2017 Aug 22,12:00:00 (America/Phoenix),2017-08-22 14:00:00 -0500,canopy_height,...,35.484918,NaN,NaN,370.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for index, row in df.iterrows():
        if row['trait'] == 'flag_leaf_emergence_time':
            df.loc[index, ['flag_leaf_emergence_time']] = row['value']

In [18]:
df.drop(labels='flag_leaf_emergence_rate', axis=1, inplace=True)

In [19]:
df.head()

,sitename,site_id,id,lat,lon,cultivar_id,date,time,raw_date,trait,...,leaf_temperature_differential,surface_temperature,aboveground_dry_biomass,seedling_emergence_rate,canopy_height,flowering_time,grain_stage_time,canopy_cover,emergence_count,flag_leaf_emergence_time
0,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011659,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,ambient_humidity,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
1,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011662,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,proximal_air_temperature,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
2,MAC Field Scanner Season 4 Range 26 Column 15,6000005593,6002011666,33.075446,-111.974819,6000000851,2017 Aug 18,12:47:00 (America/Phoenix),2017-08-18 14:47:00 -0500,leaf_temperature_differential,...,-8.589999,36.933588,NaN,NaN,348.0,NaN,NaN,NaN,NaN,NaN
3,MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012219,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,ambient_humidity,...,1.920002,35.327081,NaN,NaN,284.0,NaN,NaN,NaN,NaN,NaN
4,MAC Field Scanner Season 4 Range 21 Column 2,6000005361,6002012222,33.075266,-111.975031,6000000964,2017 Aug 18,13:20:00 (America/Phoenix),2017-08-18 15:20:00 -0500,proximal_air_temperature,...,1.920002,35.327081,NaN,NaN,284.0,NaN,NaN,NaN,NaN,NaN


In [20]:
df.isnull().sum()

sitename                              0
site_id                               0
id                                    0
lat                                   0
lon                                   0
cultivar_id                           0
date                                  0
time                                  0
raw_date                              0
trait                                 0
value                                 0
cultivar                              0
ambient_humidity                      0
range_location                        0
column_location                       0
proximal_air_temperature         137773
leaf_temperature_differential    137773
surface_temperature                5219
aboveground_dry_biomass          109832
seedling_emergence_rate          144548
canopy_height                     10740
flowering_time                   136237
grain_stage_time                 137231
canopy_cover                     145522
emergence_count                  144028


### Not Using This - Just Saving

In [ ]:
# Code from Rafa 

# from collections import defaultdict
# ​
# # make each trait a key, so you can make it a column later
# traits = df.trait.unique
# ​
# # create dictionary
# trait_columns_dict = {}
# ​
# # set all the traits as empty lists
# for trait in traits:
# 	trait_columns_dict[trait] = []
	
# # this is inefficient but it will work
# for trait, value in zip(df.trait, df.value):
# 	for col in traits:
# 		if trait == col:
# 			trait_columns_dict.append(value)
# 		else:
# 			trait_columns_dict.append(None)
# ​
# trait_columns_dict
# 	df